In [7]:
# for tensorflow
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow import keras

# other usefull library
import numpy as np
import os
from os.path import join as pjoin
from datetime import datetime
from sklearn.model_selection import train_test_split
import collections
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
path_drive = "gdrive/My Drive/ColabNotebooks/"

In [ ]:
def split_train_val(args, per_val=0.1):
    """
    Separa a base em treino e validação
    :parâmetro args: argumentos 
    :parâmetro per_val: representar a proporção do conjunto de dados a ser incluído na divisão de validação (entre 0,0 e 1,0)
    """
    # create inline and crossline sections for training and validation:
    loader_type = 'section'
    labels = np.load(pjoin((path_drive + 'data'), 'train', 'train_labels.npy'))
    
    i_list = list(range(labels.shape[0]))
    i_list = ['i_'+str(inline) for inline in i_list]
    
    x_list = list(range(labels.shape[1]))
    x_list = ['x_'+str(crossline) for crossline in x_list]
    
    list_train_val = i_list + x_list
    
    # create train and test splits:
    list_train, list_val = train_test_split(
        list_train_val, test_size=per_val, shuffle=True)

    #write to files to disK:
    file_object = open(pjoin((path_drive + 'data'), 'splits', loader_type + '_train_val.txt'), 'w')
    file_object.write('\n'.join(list_train_val))
    file_object.close()
    file_object = open(pjoin((path_drive + 'data'), 'splits', loader_type + '_train.txt'), 'w')
    file_object.write('\n'.join(list_train))
    file_object.close()
    file_object = open(pjoin((path_drive + 'data'), 'splits', loader_type + '_val.txt'), 'w')
    file_object.write('\n'.join(list_val))
    file_object.close()


In [ ]:
class section_loader():
	"""
	Data loader for the section-based deconvnet
	"""
	def __init__(self, split='train', is_transform=True, augmentations=None):      
		"""
		Inicializa os parâmetros da classe
		:parâmetro split: argumentos 
		:parâmetro is_transform: argumentos 
		:parâmetro augmentations:  
		"""      
		data = path_drive + 'data/' #path
		self.root = data
		self.split = split
		self.is_transform = is_transform
		self.augmentations = augmentations
		self.n_classes = 6 
		self.mean = 0.000941 # average of the training data  
		self.sections = collections.defaultdict(list)

		if 'test' not in self.split: 
			# Normal train/val mode
			self.seismic = np.load(pjoin((path_drive + 'data'),'train','train_seismic.npy'))
			self.labels = np.load(pjoin((path_drive + 'data'),'train','train_labels.npy'))
		elif 'test1' in self.split:
			self.seismic = np.load(pjoin((path_drive + 'data'),'test_once','test1_seismic.npy'))
			self.labels = np.load(pjoin((path_drive + 'data'),'test_once','test1_labels.npy'))
		elif 'test2' in self.split:
			self.seismic = np.load(pjoin((path_drive + 'data'),'test_once','test2_seismic.npy'))
			self.labels = np.load(pjoin((path_drive + 'data'),'test_once','test2_labels.npy'))
		else:
			raise ValueError('Unknown split.')

		if 'test' not in self.split:
			# We are in train/val mode. Most likely the test splits are not saved yet, 
			# so don't attempt to load them.  
			for split in ['train', 'val', 'train_val']:
				# reading the file names for 'train', 'val', 'trainval'""
				path = pjoin((path_drive + 'data'), 'splits', 'section_' + split + '.txt')
				file_list = tuple(open(path, 'r'))
				file_list = [id_.rstrip() for id_ in file_list]
				self.sections[split] = file_list
		elif 'test' in split:
			# We are in test mode. Only read the given split. The other one might not 
			# be available. 
			path = pjoin((path_drive + 'data'), 'splits', 'section_' + split + '.txt')
			file_list = tuple(open(path,'r'))
			file_list = [id_.rstrip() for id_ in file_list]
			self.sections[split] = file_list
		else:
			raise ValueError('Unknown split.')


	def __len__(self):
		return len(self.sections[self.split])

	def __getitem__(self, index):

		section_name = self.sections[self.split][index]
		direction, number = section_name.split(sep='_')

		if direction == 'i':
				im = self.seismic[int(number),:,:]
				lbl = self.labels[int(number),:,:]
		elif direction == 'x':    
				im = self.seismic[:,int(number),:]
				lbl = self.labels[:,int(number),:]
		
		if self.augmentations is not None:
				im, lbl = self.augmentations(im, lbl)
				
		if self.is_transform:
				im, lbl = self.transform(im, lbl)
		return im, lbl


	def transform(self, img, lbl):
		"""
		Transforma os dados na posição correta e normaliza
		:parâmetro img: dados da imagem  
		:parâmetro lbl: dados das labels 
		return: img(imagem), lbl(label)
		""" 
		img -= self.mean

		# to be in the BxCxHxW: 
		img, lbl = img.T, lbl.T

		img = np.array(img)
		lbl = np.array(lbl)

		#normalização 0-1 (os dados estão entre 1 a -1)
		scaler = MinMaxScaler(feature_range=(0,1))
		scaler = scaler.fit(img)
		img = scaler.fit_transform(img)
						
		return img, lbl

In [ ]:
def processWeights(image_label):
    """
    Contabiliza os valores dos pixels por classe para definir o peso
    :parâmetro image_label: imagem com as labels 
    return: weights (pesos de cada classe na imagem)
    """ 
    image_ = image_label
    (unique, counts) = np.unique(image_, return_counts=True)
    #weights for each class
    num_pixels = image_.shape[0] * image_.shape[1] 
    weights_ = [0,0,0,0,0,0]
    for i in range(len(counts)):
        weights_[i] = counts[i]/num_pixels
    
    return weights_

In [ ]:
def resizePotencia2(image, size, flag):
	"""
	Redefine o tamanho da imagem
	:parâmetro image: imagem
	:parâmetro size: 
	:parâmetro flag: tipo de redefinição
					- 'preencher': preenche com valores próximos
					- 'rezize': função do tensorflow que redefine a imagem
					- '': Copia a imagem para uma imagem quadrada adicionando a classe 0 de background
	return: matriz 
	""" 
	if flag == 'resize':
		return tf.image.resize(image, [size,size])

	image_ = image
	result = np.zeros((size, size))
	result[:image_.shape[0],:image_.shape[1]] = image_[:,:]

	if flag == 'preencher':
		for i in range(0, size):
			for j in range(image_.shape[1]-1, size):
				result[i][j] = result[i][j-1]

		for i in range(image_.shape[0]-1, size):
			for j in range(0, size):
				result[i][j] = result[i-1][j]

	return result

In [ ]:
def buildingSet(examples_set):
    # construção de um conjunto, vetor de imagens e labels e seus pesos
    samples = []
    samples_labels = []
    count = 0
    for i, (imagens, labels) in enumerate(examples_set):
        image_original, labels_original = imagens, labels

        # Contabiliza a proporção de pixels de casa classe
        weights_ = processWeights(labels_original)
        for j in range(len(weights_class)):                
            weights_class[j] = weights_class[j] + weights_[j] 
    
        count = count + 1 

        # image_original = resizePotencia2(image_original, 512, 'resize')
        # labels_original = resizePotencia2(labels_original, 512, 'resize')

        image_original = np.expand_dims(image_original, axis=-1) 
        labels_original = np.expand_dims(labels_original, axis=-1)

        image_original = tf.image.resize(image_original, [256,256])
        labels_original = tf.image.resize(labels_original, [256,256])  

        samples.append(image_original)
        samples_labels.append(labels_original)

    for j in range(len(weights_class)):                
        weights_class[j] = weights_class[j]/count

    return np.array(samples), np.array(samples_labels), np.array(weights_class)

In [ ]:
def tfDataset(dataset_samples, dataset_labels, num_classes, SHUFFLE_BUFFER_SIZE, BATCH_SIZE):

    # Divide a imagem da label em 6 referente as classes (uma máscara para cada classe)
    dataset_labels = tf.keras.utils.to_categorical(dataset_labels, num_classes=num_classes)
    #Tensor com o conjunto de dados X e Y
    dataset = tf.data.Dataset.from_tensor_slices((dataset_samples, dataset_labels))
    #mantém um buffer size elementos e seleciona aleatoriamente o próximo elemento desse buffer
    dataset = dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

    return dataset

## Fluxo da construção dos dados de treinamento

In [ ]:
# Argumentos
path_drive = "gdrive/My Drive/ColabNotebooks/"
BATCH_SIZE = 8
SHUFFLE_BUFFER_SIZE = 16
num_classes = 6
weights_class = [0,0,0,0,0,0]

args = {
              "arch": 'section_deconvnet',
              "n_epoch": 61,
              "batch_size": 8,
              "resume": None,
              "clip": 0.1,
              "per_val": 0.1,
              "pretrained": False,
              "aug": True,
              "class_weights": False,
            }

"""1. Load the dataset: Generate the train and validation sets for the model"""
#split_train_val(args, per_val=args['per_val'])

"""2. Load the dataset: object set """
#object to train
train_set = section_loader(is_transform=True, split='train',)    
    
# object to validation
val_set = section_loader(is_transform=True, split='val',)

"""3. Dataset: building Set """
#construção do vetor de imagens de treino
train_samples, train_labels, weights_class = buildingSet(train_set)
train_samples, train_labels = train_samples[0:200,:,:], train_labels[0:200,:,:]

#construção do vetor de imagens de validação
val_samples, val_labels, weights_class_validation = buildingSet(val_set)
val_samples, val_labels = val_samples[0:20,:,:], val_labels[0:20,:,:]

"""4. Dataset: tf dataset """
# train
train_dataset = tfDataset(train_samples, train_labels, num_classes, SHUFFLE_BUFFER_SIZE, BATCH_SIZE)
# Validation
validation_dataset = tfDataset(val_samples, val_labels, num_classes, SHUFFLE_BUFFER_SIZE, BATCH_SIZE)

In [ ]:
#Visualizar informações dos dados de treino
print("Dados de treino")
print("Samples e labels: ", train_samples.shape, train_labels.shape)
print("Dados de validação")
print("Samples e labels: ",  val_samples.shape, val_labels.shape)
print("Pesos: " , weights_class)

fig, ax = plt.subplots(1,2)
ax[0].imshow(np.array(train_samples[0][:,:,0]))
ax[1].imshow(np.array(train_labels[0][:,:,0]))
plt.show()

In [ ]:
#Visualizar informações dos dados TF de treino
print("TF dataset of train: ", train_dataset)
print("TF dataset of validation: ", validation_dataset)

In [ ]:
# deletar dados que não serão mais usados: objetos com os conjuntos dos dados
del train_samples, train_labels, val_samples, val_labels, weights_class_validation

# Rede Neural

### Encoder / Decoder

In [ ]:
def unet_model(input_shape, n_classes):
	""" Define o conjuntode treino e validação

	Parameters:
		input_shape ( ): Shape de entrada (imagens)
		n_classes (Integer): Quantidade de classes

	Returns: 
		Object: Modelo da rede
	"""

	inputs = tf.keras.layers.Input(input_shape)
	x = inputs

	# ------------------- Encoder ------------------- #

	# Block encoder 1
	conv_enc_1 = tf.keras.layers.Conv2D(64, kernel_size=3, strides=1, padding="same", activation= "relu")(x)
	conv_enc_1 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_enc_1)
	conv_enc_1 = tf.keras.layers.Conv2D(64, kernel_size=3, strides=1, padding="same", activation= "relu")(conv_enc_1)
	conv_enc_1 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_enc_1)
	max_pool_enc_1 = tf.keras.layers.MaxPool2D(pool_size=(2,2))(conv_enc_1)

	# Block encoder 2
	conv_enc_2 = tf.keras.layers.Conv2D(128, kernel_size=3, strides=1, padding="same", activation= "relu")(max_pool_enc_1)
	conv_enc_2 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_enc_2)
	conv_enc_2 = tf.keras.layers.Conv2D(128, kernel_size=3, strides=1, padding="same", activation= "relu")(conv_enc_2)
	conv_enc_2 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_enc_2)
	max_pool_enc_2 = tf.keras.layers.MaxPool2D(pool_size=(2,2))(conv_enc_2)

	# Block  encoder 3
	conv_enc_3 = tf.keras.layers.Conv2D(256, kernel_size=3, strides=1, padding="same", activation= "relu")(max_pool_enc_2)
	conv_enc_3 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_enc_3)
	conv_enc_3 = tf.keras.layers.Conv2D(256, kernel_size=3, strides=1, padding="same", activation= "relu")(conv_enc_3)
	conv_enc_3 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_enc_3)
	conv_enc_3 = tf.keras.layers.Conv2D(256, kernel_size=3, strides=1, padding="same", activation= "relu")(conv_enc_3)
	conv_enc_3 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_enc_3)
	max_pool_enc_3 = tf.keras.layers.MaxPool2D(pool_size=(2,2))(conv_enc_3)

	# Block  encoder 4
	conv_enc_4 = tf.keras.layers.Conv2D(512, kernel_size=3, strides=1, padding="same", activation= "relu")(max_pool_enc_3)
	conv_enc_4 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_enc_4)
	conv_enc_4 = tf.keras.layers.Conv2D(512, kernel_size=3, strides=1, padding="same", activation= "relu")(conv_enc_4)
	conv_enc_4 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_enc_4)
	conv_enc_4 = tf.keras.layers.Conv2D(512, kernel_size=3, strides=1, padding="same", activation= "relu")(conv_enc_4)
	max_pool_enc_4 = tf.keras.layers.MaxPool2D(pool_size=(2,2))(conv_enc_4)

	# Block  encoder 5
	conv_enc_5 = tf.keras.layers.Conv2D(512, kernel_size=3, strides=1, padding="same", activation= "relu")(max_pool_enc_4)
	conv_enc_5 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_enc_5)
	conv_enc_5 = tf.keras.layers.Conv2D(512, kernel_size=3, strides=1, padding="same", activation= "relu")(conv_enc_5)
	conv_enc_5 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_enc_5)
	conv_enc_5 = tf.keras.layers.Conv2D(512, kernel_size=3, strides=1, padding="same", activation= "relu")(conv_enc_5)
	max_pool_enc_5 = tf.keras.layers.MaxPool2D(pool_size=(2,2))(conv_enc_5)

	# Block  encoder 6
	conv_enc_6 = tf.keras.layers.Conv2D(4096, kernel_size=3, strides=1, padding="same", activation= "relu")(max_pool_enc_5)
	conv_enc_6 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_enc_6)

	# Block  encoder 7
	conv_enc_7 = tf.keras.layers.Conv2D(4096, kernel_size=3, strides=1, padding="same", activation= "relu")(conv_enc_6)
	conv_enc_7 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_enc_7)
	

	# ------------------- Decoder ------------------- #

	# Block  decoder 8
	conv_dec_8 = tf.keras.layers.Conv2DTranspose(4096, kernel_size=3, strides=1, padding="same", activation= "relu")(conv_enc_7)
	conv_dec_8 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_dec_8)

	# Block  decoder 9
	up_dec_9 = tf.keras.layers.UpSampling2D(2)(conv_dec_8)
	up_dec_9 = tf.keras.layers.Concatenate()([up_dec_9, conv_enc_5])

	# Block  decoder 10
	conv_dec_10 = tf.keras.layers.Conv2DTranspose(512, kernel_size=3, strides=1, padding="same", activation= "relu")(up_dec_9)
	conv_dec_10 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_dec_10)
	conv_dec_10 = tf.keras.layers.Conv2DTranspose(512, kernel_size=3, strides=1, padding="same", activation= "relu")(conv_dec_10)
	conv_dec_10 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_dec_10)
	conv_dec_10 = tf.keras.layers.Conv2DTranspose(512, kernel_size=3, strides=1, padding="same", activation= "relu")(conv_dec_10)
	conv_dec_10 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_dec_10)

	# Block  decoder 11
	up_dec_11 = tf.keras.layers.UpSampling2D(2)(conv_dec_10)
	up_dec_11 = tf.keras.layers.Concatenate()([up_dec_11, conv_enc_4])

	# Block  decoder 12
	conv_dec_12 = tf.keras.layers.Conv2DTranspose(512, kernel_size=3, strides=1, padding="same", activation= "relu")(up_dec_11)
	conv_dec_12 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_dec_12)
	conv_dec_12 = tf.keras.layers.Conv2DTranspose(512, kernel_size=3, strides=1, padding="same", activation= "relu")(conv_dec_12)
	conv_dec_12 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_dec_12)
	conv_dec_12 = tf.keras.layers.Conv2DTranspose(256, kernel_size=3, strides=1, padding="same", activation= "relu")(conv_dec_12)
	conv_dec_12 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_dec_12)

	# Block  decoder 13
	up_dec_13 = tf.keras.layers.UpSampling2D(2)(conv_dec_12)
	up_dec_13 = tf.keras.layers.Concatenate()([up_dec_13, conv_enc_3])

	# Block  decoder 14
	conv_dec_14 = tf.keras.layers.Conv2DTranspose(256, kernel_size=3, strides=1, padding="same", activation= "relu")(up_dec_13)
	conv_dec_14 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_dec_14)
	conv_dec_14 = tf.keras.layers.Conv2DTranspose(256, kernel_size=3, strides=1, padding="same", activation= "relu")(conv_dec_14)
	conv_dec_14 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_dec_14)
	conv_dec_14 = tf.keras.layers.Conv2DTranspose(128, kernel_size=3, strides=1, padding="same", activation= "relu")(conv_dec_14)
	conv_dec_14 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_dec_14)

	# Block  decoder 15
	up_dec_15 = tf.keras.layers.UpSampling2D(2)(conv_dec_14)
	up_dec_15 = tf.keras.layers.Concatenate()([up_dec_15, conv_enc_2])

	# Block  decoder 16
	conv_dec_16 = tf.keras.layers.Conv2DTranspose(128, kernel_size=3, strides=1, padding="same", activation= "relu")(up_dec_15)
	conv_dec_16 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_dec_16)
	conv_dec_16 = tf.keras.layers.Conv2DTranspose(128, kernel_size=3, strides=1, padding="same", activation= "relu")(conv_dec_16)
	conv_dec_16 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_dec_16)

	# Block  decoder 17
	up_dec_17 = tf.keras.layers.UpSampling2D(2)(conv_dec_16)
	up_dec_17 = tf.keras.layers.Concatenate()([up_dec_17, conv_enc_1])

	# Block  decoder 18
	conv_dec_18 = tf.keras.layers.Conv2DTranspose(64, kernel_size=3, strides=1, padding="same", activation= "relu")(up_dec_17)
	conv_dec_18 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_dec_18)
	conv_dec_18 = tf.keras.layers.Conv2DTranspose(64, kernel_size=3, strides=1, padding="same", activation= "relu")(conv_dec_18)
	conv_dec_18 = tf.keras.layers.BatchNormalization(momentum=0.1,epsilon=1e-05)(conv_dec_18)

	# Block  decoder 19
	output = tf.keras.layers.Conv2D(n_classes, kernel_size=1, padding="same", activation='softmax')(conv_dec_18)  

	model_unet = tf.keras.models.Model(inputs, output)
	model_unet.summary()

	return model_unet